# OpenRefine MCO2

This dataset mainly focuses on tides in Puerto Princesa, Palawan from 2017 to 2020. 
The information that will be included in this dataset are:
- date of record
- day of record
- sunrise time
- sunset time
- moonrise time
- moonset time
- tide height
- tide time
- tidal coefficient
- coefficient interpretation
- red tide occurrence in Honda bay and Puerto Princesa Bay

# Import of Packages

In [7]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time as time
import json

import os
from IPython.display import display

# Data Collection (NO NEED TO RUN)
The sites that were scraped are as follows:
- Tide Information:
    - https://tides4fishing.com/as/philippines/puerto-princesa#_tide_table
- Moon Rise and Set
    - http://moon.5deg.com/ph/puerto-princesa/
    
Information on the red tide occurrences were obtained from the red tide bulletin of the Philippine Burueau of Fisheries and Aquatic Resources and was manually encoded into a csv file.

## Tide Information
Information about the tides were obtained from Tides4Fishing. The site's content for historical data could not be accessed through changing the URL since the site was coded in such a way that the data is being accessed through a javascript within the site. Selenium was used to scrape the historical data.

Accessing the Tides4Fishing site and checking of page content

In [ ]:
URL = "https://tides4fishing.com/as/philippines/puerto-princesa#_tide_table"

#This variable holds the information requested from the URL
page = requests.get(URL)

#This variable gets the HTML code through page
soup = BeautifulSoup(page.content, 'html.parser') 
soup

Finding and accessing the table where the desired data was scraped.

In [ ]:
# From the HTML code stored in soup, we find the portion of the code with id, tabla_mareas_fondo.
# In which, we will be looking for the desired table.
tides4fishing_content = soup.find(id = 'tabla_mareas_fondo')
tides4fishing_tables = tides4fishing_content.find_all('table')

Trial of scraping of one month and temporarily placing the information obtained into a data frame.

Information can be accessed through the following indices:

|Index|Information|
|-----|-----------|
|  1  |Day of Month (1-31) and Day of Week (Sun, Mon, Tue, Wed, Thu, Fri, Sat)|
|  5  |Sunrise and Sunset Time (24-hour format)|
|  7  |1st Tide (Time and Height in m)|
|  9  |2nd Tide (Time and Height in m)|
| 11  |3rd Tide (Time and Height in m)|
| 13  |4th Tide (Time and Height in m)|
| 15  |Tidal Coefficient|


Complete scraping of all the data through Selenium and saving each month into a JSON file.

In [9]:
options = Options()
options.add_argument('headless');
options.headless = True
options.add_argument("--window-size=1920,1200")
jsonlist=[]

DRIVER_PATH = r'C:/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://tides4fishing.com/as/philippines/puerto-princesa')
#time.sleep(15)

capabilities = {
 "resolution": "1920x1080"
}

driver.find_element_by_xpath("//*[@id='botones_cookies']/a[2]").click()

for x in range(54):
    info_start = 6
    t4f_json = []
    filename = 't4f'
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    tides4fishing_content = soup.find(id='tabla_mareas_fondo')
    tides4fishing_tables = tides4fishing_content.find_all('table')
    #t4ftable=driver.find_element_by_xpath("//table[@id='tabla_mareas']")
    #t4ftable= soup.find_all('table', attrs={"id":"tabla_mareas"})
    t4f = tides4fishing_tables[0]
    t4f_info = t4f.contents[1]
    
    
    for i in range(info_start, len(t4f_info.contents), 4):
        tidetime1 = "no record"
        tideheight1 = "no record"
        tidetime2 = "no record"
        tideheight2 = "no record"
        tidetime3 = "no record"
        tideheight3 = "no record"
        tidetime4 = "no record"
        tideheight4 = "no record"
        
        
        data = t4f_info.contents[i]
        day = " ".join(data.contents[1].text.strip().split())
        time =" ".join(data.contents[5].text.strip().split())
        firsttide = " ".join(data.contents[7].text.strip().split())
        secondtide = " ".join(data.contents[9].text.strip().split())
        thirdtide = " ".join(data.contents[11].text.strip().split())
        fourthtide = " ".join(data.contents[13].text.strip().split())
        coefficient = " ".join(data.contents[15].text.strip().split())
        
        firsttide = firsttide.split()
        if len(firsttide) == 4:
            tidetime1 = firsttide[0] + " " + firsttide[1]
            tideheight1 = firsttide[2] + " " + firsttide[3]
        
        secondtide = secondtide.split()
        if len(secondtide) == 4:
            tidetime2 = secondtide[0] + " " + secondtide[1]
            tideheight2 = secondtide[2] + " " + secondtide[3]
        
        thirdtide = thirdtide.split()
        if len(thirdtide) == 4:
            tidetime3 = thirdtide[0] + " " + thirdtide[1]
            tideheight3 = thirdtide[2] + " " + thirdtide[3]
        
        fourthtide = fourthtide.split()
        if len(fourthtide) == 4:
            tidetime4 = fourthtide[0] + " " + fourthtide[1]
            tideheight4 = fourthtide[2] + " " + fourthtide[3]
        
        t4f_json.append({
            "day": day,
            "time": time,
            "tidetime1": tidetime1,
            "tideheight1": tideheight1,
            "tidetime2": tidetime2,
            "tideheight2": tideheight2,
            "tidetime3": tidetime3,
            "tideheight3": tideheight3,
            "tidetime4":tidetime4,
            "tideheight4":tideheight4,
            "coefficient": coefficient
            })
        
    #print(day)
    #dataframe = pd.DataFrame.from_dict(t4f_json)
    #dataframe.reset_index(drop=True)
    #dataframe
    filenum = str(x)
    formatedjson = json.dumps(t4f_json, indent = 4)
    with open(filename+ '-' + filenum+'.json', "w") as outfile:
        outfile.write(formatedjson)
    
    back = driver.find_element_by_xpath("//*[@id='tabla_mareas_mes']/a[1]")    
    #wait = WebDriverWait(driver, 60);
    #wait.until(EC.invisibilityOfElementLocated(back));
    back.click()
    #time.sleep(10)
    #WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='tabla_mareas_mes']/a[1]"))).click()    
    #back= driver.find_element_by_xpath("//*[@id='tabla_mareas_mes']/a[1]")
    


t4f_json    
#driver.quit()

[{'day': '1 Thu',
  'time': '6:06 h 17:42 h',
  'tidetime1': '6:25 h',
  'tideheight1': '-0.1 m',
  'tidetime2': '13:32 h',
  'tideheight2': '0.7 m',
  'tidetime3': '14:52 h',
  'tideheight3': '0.4 m',
  'tidetime4': '22:50 h',
  'tideheight4': '1.4 m',
  'coefficient': '77 high'},
 {'day': '2 Fri',
  'time': '6:06 h 17:43 h',
  'tidetime1': '7:05 h',
  'tideheight1': '-0.1 m',
  'tidetime2': '23:21 h',
  'tideheight2': '1.4 m',
  'tidetime3': 'no record',
  'tideheight3': 'no record',
  'tidetime4': 'no record',
  'tideheight4': 'no record',
  'coefficient': '73 high'},
 {'day': '3 Sat',
  'time': '6:07 h 17:43 h',
  'tidetime1': '7:47 h',
  'tideheight1': '-0.1 m',
  'tidetime2': '23:54 h',
  'tideheight2': '1.4 m',
  'tidetime3': 'no record',
  'tideheight3': 'no record',
  'tidetime4': 'no record',
  'tideheight4': 'no record',
  'coefficient': '67 average'},
 {'day': '4 Sun',
  'time': '6:07 h 17:43 h',
  'tidetime1': '8:30 h',
  'tideheight1': '-0.1 m',
  'tidetime2': 'no record'

## Moonrise and Moonset

Accessing of Moon.5deg site. All tables with tableline as class within the HTML was obtained. Checking of index where the desired table of values was also inspected. Desired table is located at index 1.

The table below shows the index of the information from the desired table.

|Index|Information|
|-----|-----------|
|  1  |Day of Month (1-31)|
|  3  |Day of Week (Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday)|
|  5  |Moonrise Time (24-hour format)|
|  7  |Moonset Time (24-hour format)|
|  11 |Approximate Distance of Moon from Earth in km|

In [ ]:
URL_List = ["http://moon.5deg.com/ph/puerto-princesa/2017/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2017/12/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2018/12/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2019/12/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/1/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/2/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/3/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/4/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/5/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/6/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/7/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/8/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/9/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/10/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/11/",
            "http://moon.5deg.com/ph/puerto-princesa/2020/12/"]

moon = []

for url in URL_List:
    #This variable holds the information requested from the URL
    page = requests.get(url)
    
    #This variable gets the HTML code through page
    soup = BeautifulSoup(page.content, 'html.parser')
    table_content = soup.find_all('table', {"class" : "tableline"})

    #Getting the rows from the table
    moon_table = table_content[1]
    moon_rows = moon_table.find_all('tr')
    moon_rows

    yearMonth = url.split('/')

    for r in range(1, len(moon_rows)):
        date = yearMonth[6] + '/' + moon_rows[r].contents[1].text.strip() + '/' + yearMonth[5]
        day = moon_rows[r].contents[3].text.strip()
        moonrise = moon_rows[r].contents[5].text.strip()
        moonset = moon_rows[r].contents[7].text.strip()
        distance = moon_rows[r].contents[11].text.strip().split(" ")[0]

        moon.append({
                "date": date,
                "day": day,
                "moonrise": moonrise,
                "moonset": moonset,
                "distance": distance
            })

    print("Successful scraping for " + yearMonth[5] + "/" + yearMonth[6])
    
#This is to convert the JSON string from moon to a compatible JSON format using double quotes instead of single quotes
moonjson = json.dumps(moon)
    
#Generate JSON file for all dates
f = open("moon.json", "w")
json_str = str(moonjson)
f.write(json_str)
f.close()

# Opening and Processing of JSON and CSV files to Dataframes

## Tides4Fishing Data

In [8]:
months = ['December','November','October','September','August', 'July', 'June','May', 'April', 'March', 'February','January']
months_num = [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
twelvetracker = 0
filename = 't4f'
year = 2021
datalist = []
dataframelist = []

#Opening of monthly tide information JSON files (from December 2020 to January 2017) and appending them to datalist
for x in range(5, 53):
    filenum = str(x)
    with open('v2/t4f jsons/' + filename+ '-' + filenum+'.json', 'r') as openfile:
        t4f_json = json.load(openfile)
    datalist.append(t4f_json)
    
#Converting each JSON file to a dataframe
for k in range(48):
    dataframelist.append(pd.DataFrame.from_dict(datalist[k]))

#Processing and printing of dataframes
for i in dataframelist:
    if twelvetracker == 0:
        year = year - 1
        stryear = str(year)
    else:
        stryear = str(year)
    
    print(months[twelvetracker] + " " + stryear)
    
    #Formatting days to date
    dates = []
    dayofWeek =[]
    
    tempDays = i['day'].str.split()
    
    for j in range(len(tempDays)):
        dayofMonth = tempDays[j][0]
        dayofWeek.append(tempDays[j][1])
        date = str(months_num[twelvetracker]) + '/' + dayofMonth + '/' + stryear
        dates.append(date)

    dates
    i["date"] = dates
    i['date'] = pd.to_datetime(i['date'])
    
    #Splitting time column to sunrise and sunset columns
    sunrise = []
    sunset = []

    for s in range(len(i['time'])):
        sunrise.append(i['time'].str.split()[s][0])
        sunset.append(i['time'].str.split()[s][2])

    i["sunrise"] = sunrise
    i["sunset"] = sunset
    
    #Removing units included in the entries
    for t in range(len(i["tidetime1"])):
        if (i["tidetime1"][t] != "no record"):
            i["tidetime1"][t] = i["tidetime1"][t].split()[0]
        if (i["tideheight1"][t] != "no record"):
            i["tideheight1"][t] = i["tideheight1"][t].split()[0]
        if (i["tidetime2"][t] != "no record"):
            i["tidetime2"][t] = i["tidetime2"][t].split()[0]
        if (i["tideheight2"][t] != "no record"):
            i["tideheight2"][t] = i["tideheight2"][t].split()[0]
        if (i["tidetime3"][t] != "no record"):
            i["tidetime3"][t] = i["tidetime3"][t].split()[0]
        if (i["tideheight3"][t] != "no record"):
            i["tideheight3"][t] = i["tideheight3"][t].split()[0]
        if (i["tidetime4"][t] != "no record"):
            i["tidetime4"][t] = i["tidetime4"][t].split()[0]
        if (i["tideheight4"][t] != "no record"):
            i["tideheight4"][t] = i["tideheight4"][t].split()[0]
    
    #Splitting the coefficient column to coefficient and coeff_interpretation columns
    coefficient = []
    coeff_interpretation = []

    for c in range(len(i['coefficient'])):
        coefficient.append(i['coefficient'].str.split()[c][0])
        coeff_interpretation.append(i['coefficient'].str.split()[c][1])

    i["coefficient"] = coefficient
    i["coeff_interpretation"] = coeff_interpretation
    
    #Display the datarframes
    display(i)
    print('\n')
    
    #This is to track and update the month
    twelvetracker = twelvetracker + 1
    if twelvetracker > 11:
        twelvetracker = 0

December 2020


<ipython-input-8-961ec1916590>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-8-961ec1916590>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-8-961ec1916590>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-8-961ec1916590>:66: SettingWit

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,1 Tue,6:06 h 17:42 h,5:54,-0.1,13:15,0.7,14:09,0.4,22:17,1.4,77,2020-12-01,6:06,17:42,high
1,2 Wed,6:06 h 17:43 h,6:36,-0.2,22:46,1.4,no record,no record,no record,no record,74,2020-12-02,6:06,17:43,high
2,3 Thu,6:07 h 17:43 h,7:20,-0.2,23:18,1.4,no record,no record,no record,no record,69,2020-12-03,6:07,17:43,average
3,4 Fri,6:07 h 17:43 h,8:08,-0.2,23:53,1.4,no record,no record,no record,no record,62,2020-12-04,6:07,17:43,average
4,5 Sat,6:08 h 17:43 h,8:59,-0.2,no record,no record,no record,no record,no record,no record,56,2020-12-05,6:08,17:43,average
5,6 Sun,6:09 h 17:44 h,0:31,1.4,9:50,-0.2,no record,no record,no record,no record,51,2020-12-06,6:09,17:44,average
6,7 Mon,6:09 h 17:44 h,1:14,1.4,10:39,-0.1,no record,no record,no record,no record,50,2020-12-07,6:09,17:44,average
7,8 Tue,6:10 h 17:45 h,2:05,1.3,11:21,-0.1,no record,no record,no record,no record,55,2020-12-08,6:10,17:45,average
8,9 Wed,6:10 h 17:45 h,3:08,1.1,11:55,0.0,20:03,0.8,22:44,0.5,64,2020-12-09,6:10,17:45,average
9,10 Thu,6:11 h 17:45 h,4:39,1.0,12:20,0.1,19:42,0.9,no record,no record,74,2020-12-10,6:11,17:45,high




November 2020


<ipython-input-8-961ec1916590>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-8-961ec1916590>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-8-961ec1916590>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-8-961ec1916590>:66: SettingWit

...

<ipython-input-8-961ec1916590>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-8-961ec1916590>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-8-961ec1916590>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-8-961ec1916590>:66: SettingWit

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,1 Wed,6:27 h 18:10 h,0:19,1.2,8:01,-0.1,14:31,0.6,18:07,0.3,80,2017-02-01,6:27,18:10,high
1,2 Thu,6:27 h 18:11 h,1:00,1.1,8:25,0.0,14:57,0.7,19:27,0.3,71,2017-02-02,6:27,18:11,high
2,3 Fri,6:27 h 18:11 h,1:46,1.0,8:45,0.1,15:28,0.8,21:05,0.2,62,2017-02-03,6:27,18:11,average
3,4 Sat,6:27 h 18:11 h,2:44,0.8,9:01,0.1,16:07,0.9,23:06,0.2,56,2017-02-04,6:27,18:11,average
4,5 Sun,6:27 h 18:12 h,4:19,0.6,9:12,0.2,16:55,1.1,no record,no record,55,2017-02-05,6:27,18:12,average
5,6 Mon,6:27 h 18:12 h,1:14,0.1,7:43,0.5,8:55,0.2,17:49,1.2,60,2017-02-06,6:27,18:12,average
6,7 Tue,6:26 h 18:12 h,2:43,-0.1,18:47,1.3,no record,no record,no record,no record,70,2017-02-07,6:26,18:12,high
7,8 Wed,6:26 h 18:12 h,3:41,-0.2,19:45,1.4,no record,no record,no record,no record,81,2017-02-08,6:26,18:12,high
8,9 Thu,6:26 h 18:13 h,4:28,-0.3,20:40,1.4,no record,no record,no record,no record,90,2017-02-09,6:26,18:13,very
9,10 Fri,6:26 h 18:13 h,5:08,-0.3,21:31,1.4,no record,no record,no record,no record,96,2017-02-10,6:26,18:13,very




January 2017


<ipython-input-8-961ec1916590>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime1"][t] = i["tidetime1"][t].split()[0]
<ipython-input-8-961ec1916590>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tideheight1"][t] = i["tideheight1"][t].split()[0]
<ipython-input-8-961ec1916590>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i["tidetime2"][t] = i["tidetime2"][t].split()[0]
<ipython-input-8-961ec1916590>:66: SettingWit

,day,time,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,1 Sun,6:21 h 17:56 h,7:44,-0.2,23:44,1.4,no record,no record,no record,no record,78,2017-01-01,6:21,17:56,high
1,2 Mon,6:22 h 17:57 h,8:12,-0.2,no record,no record,no record,no record,no record,no record,73,2017-01-02,6:22,17:57,high
2,3 Tue,6:22 h 17:57 h,0:20,1.3,8:47,-0.2,no record,no record,no record,no record,68,2017-01-03,6:22,17:57,average
3,4 Wed,6:23 h 17:58 h,0:58,1.2,9:20,-0.1,no record,no record,no record,no record,62,2017-01-04,6:23,17:58,average
4,5 Thu,6:23 h 17:58 h,1:40,1.1,9:50,0.0,17:30,0.7,20:17,0.4,58,2017-01-05,6:23,17:58,average
5,6 Fri,6:23 h 17:59 h,2:31,1.0,10:16,0.0,17:44,0.8,22:48,0.4,58,2017-01-06,6:23,17:59,average
6,7 Sat,6:24 h 17:59 h,3:47,0.8,10:40,0.1,18:09,1.0,no record,no record,62,2017-01-07,6:24,17:59,average
7,8 Sun,6:24 h 18:00 h,1:05,0.2,6:03,0.6,11:00,0.2,18:43,1.1,70,2017-01-08,6:24,18:00,high
8,9 Mon,6:24 h 18:00 h,2:40,0.1,8:34,0.6,11:16,0.2,19:22,1.3,79,2017-01-09,6:24,18:00,high
9,10 Tue,6:25 h 18:01 h,3:42,-0.1,20:05,1.4,no record,no record,no record,no record,89,2017-01-10,6:25,18:01,high


Dropping the repetitive columns that were already split.

In [9]:
tidesdf = dataframelist[0]

for i in range(1, len(dataframelist)):
    tidesdf = tidesdf.append(dataframelist[i])
    
tidesdf.drop('day', inplace=True, axis=1)
tidesdf.drop('time', inplace=True, axis=1)

In [10]:
tidesdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 30
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tidetime1             1461 non-null   object        
 1   tideheight1           1461 non-null   object        
 2   tidetime2             1461 non-null   object        
 3   tideheight2           1461 non-null   object        
 4   tidetime3             1461 non-null   object        
 5   tideheight3           1461 non-null   object        
 6   tidetime4             1461 non-null   object        
 7   tideheight4           1461 non-null   object        
 8   coefficient           1461 non-null   object        
 9   date                  1461 non-null   datetime64[ns]
 10  sunrise               1461 non-null   object        
 11  sunset                1461 non-null   object        
 12  coeff_interpretation  1461 non-null   object        
dtypes: datetime64[ns](1)

In [11]:
tidesdf

,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,date,sunrise,sunset,coeff_interpretation
0,5:54,-0.1,13:15,0.7,14:09,0.4,22:17,1.4,77,2020-12-01,6:06,17:42,high
1,6:36,-0.2,22:46,1.4,no record,no record,no record,no record,74,2020-12-02,6:06,17:43,high
2,7:20,-0.2,23:18,1.4,no record,no record,no record,no record,69,2020-12-03,6:07,17:43,average
3,8:08,-0.2,23:53,1.4,no record,no record,no record,no record,62,2020-12-04,6:07,17:43,average
4,8:59,-0.2,no record,no record,no record,no record,no record,no record,56,2020-12-05,6:08,17:43,average
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,5:30,-0.2,21:47,1.3,no record,no record,no record,no record,85,2017-01-27,6:27,18:09,high
27,6:03,-0.2,13:03,0.5,14:06,0.2,22:25,1.4,89,2017-01-28,6:27,18:09,high
28,6:36,-0.2,13:21,0.5,15:04,0.2,23:02,1.3,92,2017-01-29,6:27,18:09,very
29,7:06,-0.2,13:43,0.6,15:59,0.2,23:40,1.3,90,2017-01-30,6:27,18:10,very


## Moon Data

Opening the Moon Data

In [12]:
moonframe = pd.read_json('moon.json')

In [13]:
moonframe

,date,day,moonrise,moonset,distance
0,2017-01-01,Sunday,08:29,20:31,387620
1,2017-01-02,Monday,09:16,21:23,382892
2,2017-01-03,Tuesday,10:02,22:14,378186
3,2017-01-04,Wednesday,10:48,23:06,373755
4,2017-01-05,Thursday,11:34,23:59,369848
...,...,...,...,...,...
1456,2020-12-27,Sunday,15:36,03:39,403274
1457,2020-12-28,Monday,16:22,04:29,400801
1458,2020-12-29,Tuesday,17:12,05:21,397535
1459,2020-12-30,Wednesday,18:05,06:14,393608


## Red Tide Data

Opening the Red Tide Data

In [14]:
redtideFrame = pd.read_csv('RedTide.csv')

Transforming `date` column from object to datetime format since the `date` column is not yet in datetime format.

In [15]:
redtideFrame['date'] = pd.to_datetime(redtideFrame['date'])

In [16]:
redtideFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1462 non-null   datetime64[ns]
 1   redtide_pp     1462 non-null   int64         
 2   redtide_honda  1462 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 34.4 KB


In [17]:
redtideFrame

,date,redtide_pp,redtide_honda
0,2016-12-31,0,0
1,2017-01-01,0,0
2,2017-01-02,0,0
3,2017-01-03,0,0
4,2017-01-04,0,0
...,...,...,...
1457,2020-12-27,1,1
1458,2020-12-28,1,1
1459,2020-12-29,1,1
1460,2020-12-30,1,1


# Data Merging

Merging of Moon dataframe with Red Tide dataframe through left join using `date` of Moon as key to the final dataframe.

In [18]:
finalDF = moonframe.merge(redtideFrame, how='left', on='date')

Merging of final dataframe with tide information dataframe through left join using `date` of final dataframe as key.

In [19]:
finalDF = finalDF.merge(tidesdf, how='left', on='date')

Rearranging the columns of the final dataframe.

In [20]:
finalDF = finalDF[['date', 'day', 'sunrise', 'sunset', 'moonrise', 'moonset', 'distance','tidetime1','tideheight1','tidetime2','tideheight2','tidetime3','tideheight3','tidetime4','tideheight4','coefficient','coeff_interpretation','redtide_pp','redtide_honda']]

In [21]:
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   object        
 3   sunset                1461 non-null   object        
 4   moonrise              1461 non-null   object        
 5   moonset               1461 non-null   object        
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   object        
 8   tideheight1           1461 non-null   object        
 9   tidetime2             1461 non-null   object        
 10  tideheight2           1461 non-null   object        
 11  tidetime3             1461 non-null   object        
 12  tideheight3           1461 non-null   object        
 13  tidetime4         

In [22]:
finalDF

,date,day,sunrise,sunset,moonrise,moonset,distance,tidetime1,tideheight1,tidetime2,tideheight2,tidetime3,tideheight3,tidetime4,tideheight4,coefficient,coeff_interpretation,redtide_pp,redtide_honda
0,2017-01-01,Sunday,6:21,17:56,08:29,20:31,387620,7:44,-0.2,23:44,1.4,no record,no record,no record,no record,78,high,0,0
1,2017-01-02,Monday,6:22,17:57,09:16,21:23,382892,8:12,-0.2,no record,no record,no record,no record,no record,no record,73,high,0,0
2,2017-01-03,Tuesday,6:22,17:57,10:02,22:14,378186,0:20,1.3,8:47,-0.2,no record,no record,no record,no record,68,average,0,0
3,2017-01-04,Wednesday,6:23,17:58,10:48,23:06,373755,0:58,1.2,9:20,-0.1,no record,no record,no record,no record,62,average,0,0
4,2017-01-05,Thursday,6:23,17:58,11:34,23:59,369848,1:40,1.1,9:50,0.0,17:30,0.7,20:17,0.4,58,average,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2020-12-27,Sunday,6:19,17:53,15:36,03:39,403274,3:51,0.0,10:10,0.6,12:25,0.3,20:20,1.2,67,average,1,1
1457,2020-12-28,Monday,6:20,17:54,16:22,04:29,400801,4:34,-0.1,20:51,1.3,no record,no record,no record,no record,72,high,1,1
1458,2020-12-29,Tuesday,6:20,17:54,17:12,05:21,397535,5:14,-0.2,21:24,1.4,no record,no record,no record,no record,76,high,1,1
1459,2020-12-30,Wednesday,6:21,17:55,18:05,06:14,393608,5:54,-0.2,21:59,1.4,no record,no record,no record,no record,78,high,1,1


# Further Data Processing

Converting the datatype of coefficient and tide heights from object to its most appropriate numeric datatypes.

In [23]:
finalDF['coefficient'] = pd.to_numeric(finalDF['coefficient'], errors='coerce')
finalDF['tideheight1'] = pd.to_numeric(finalDF['tideheight1'], errors='coerce')
finalDF['tideheight2'] = pd.to_numeric(finalDF['tideheight2'], errors='coerce')
finalDF['tideheight3'] = pd.to_numeric(finalDF['tideheight3'], errors='coerce')
finalDF['tideheight4'] = pd.to_numeric(finalDF['tideheight4'], errors='coerce')

Converting the datatype of sunrise, sunset, moonrise, and moonset times from object to datetime format.

In [24]:
#Concatenating the corresponding date of the entry with the time of whatever it represents
for i in range(len(finalDF['sunrise'])):
    # sunrise time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['sunrise'][i]
    time = pd.to_datetime(time)
    finalDF['sunrise'][i] = time
    # sunset time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['sunset'][i]
    time = pd.to_datetime(time)
    finalDF['sunset'][i] = time
    # moonrise time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['moonrise'][i]
    time = pd.to_datetime(time)
    finalDF['moonrise'][i] = time
    # moonset time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['moonset'][i]
    time = pd.to_datetime(time)
    finalDF['moonset'][i] = time
    
#Conversion proper
finalDF['sunrise'] =pd.to_datetime(finalDF['sunrise'])
finalDF['sunset'] =pd.to_datetime(finalDF['sunset'])
finalDF['moonrise'] =pd.to_datetime(finalDF['moonrise'])
finalDF['moonset'] =pd.to_datetime(finalDF['moonset'])

<ipython-input-24-d9604eb6a25a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['sunrise'][i] = time
<ipython-input-24-d9604eb6a25a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['sunset'][i] = time
<ipython-input-24-d9604eb6a25a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['moonrise'][i] = time
<ipython-input-24-d9604eb6a25a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [25]:
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   datetime64[ns]
 3   sunset                1461 non-null   datetime64[ns]
 4   moonrise              1461 non-null   datetime64[ns]
 5   moonset               1461 non-null   datetime64[ns]
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   object        
 8   tideheight1           1461 non-null   float64       
 9   tidetime2             1461 non-null   object        
 10  tideheight2           1411 non-null   float64       
 11  tidetime3             1461 non-null   object        
 12  tideheight3           811 non-null    float64       
 13  tidetime4         

Converting the datatype of tide times from object to datetime format.

In [26]:
#Concatenating and converting
for i in range(len(finalDF['tidetime1'])):
    # tide 1 time conversion to datetime data type
    time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime1'][i]
    time = pd.to_datetime(time)
    finalDF['tidetime1'][i] = time
    # tide 2 time conversion to datetime data type
    if (finalDF['tidetime2'][i] != "no record"):
        time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime2'][i]
        time = pd.to_datetime(time)
        finalDF['tidetime2'][i] = time
    # tide 3 time conversion to datetime data type
    if (finalDF['tidetime3'][i] != "no record"):
        time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime3'][i]
        time = pd.to_datetime(time)
        finalDF['tidetime3'][i] = time
    # tide 4 time conversion to datetime data type
    if (finalDF['tidetime4'][i] != "no record"):
        time = str(finalDF['date'][i]).split()[0] + " " + finalDF['tidetime4'][i]
        time = pd.to_datetime(time)
        finalDF['tidetime4'][i] = time
        
#To fill in no records with NaT
finalDF['tidetime1'] = pd.to_datetime(finalDF['tidetime1'], errors='coerce')
finalDF['tidetime2'] = pd.to_datetime(finalDF['tidetime2'], errors='coerce')
finalDF['tidetime3'] = pd.to_datetime(finalDF['tidetime3'], errors='coerce')
finalDF['tidetime4'] = pd.to_datetime(finalDF['tidetime4'], errors='coerce')

<ipython-input-26-843d2dc4cee3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetime1'][i] = time
<ipython-input-26-843d2dc4cee3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetime2'][i] = time
<ipython-input-26-843d2dc4cee3>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetime3'][i] = time
<ipython-input-26-843d2dc4cee3>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [27]:
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   datetime64[ns]
 3   sunset                1461 non-null   datetime64[ns]
 4   moonrise              1461 non-null   datetime64[ns]
 5   moonset               1461 non-null   datetime64[ns]
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   datetime64[ns]
 8   tideheight1           1461 non-null   float64       
 9   tidetime2             1411 non-null   datetime64[ns]
 10  tideheight2           1411 non-null   float64       
 11  tidetime3             811 non-null    datetime64[ns]
 12  tideheight3           811 non-null    float64       
 13  tidetime4         

Getting the tide type (high or low tide) based on the given tide heights. 

In [28]:
tidetype1 = []
tidetype2 = []
tidetype3 = []
tidetype4 = []

#Populating the arrays with placeholders
for i in range(len(finalDF['date'])):
    tidetype1.append('-')
    tidetype2.append('-')
    tidetype3.append('-')
    tidetype4.append('-')

#Placing information of the arrays into columns in the final dataframe
finalDF['tidetype1'] = tidetype1
finalDF['tidetype2'] = tidetype2
finalDF['tidetype3'] = tidetype3
finalDF['tidetype4'] = tidetype4

#Replacing the placeholders with actual values (tide type)
# 0 = no record, -1 = low, 1 = high
for i in range(len(finalDF['date'])):
    #If only one tide was recorded for the day
    if(pd.notnull(finalDF['tideheight1'][i]) and pd.isnull(finalDF['tideheight2'][i])):
        if(finalDF['tideheight1'][i] > 0):
            finalDF['tidetype1'][i] = 1
            finalDF['tidetype2'][i] = 0
            finalDF['tidetype3'][i] = 0
            finalDF['tidetype4'][i] = 0
        elif(finalDF['tideheight1'][i] <= 0):
            finalDF['tidetype1'][i] = -1
            finalDF['tidetype2'][i] = 0
            finalDF['tidetype3'][i] = 0
            finalDF['tidetype4'][i] = 0
    #If two or more tides were recorded for the day
    elif(pd.notnull(finalDF['tideheight1'][i]) and pd.notnull(finalDF['tideheight2'][i])):
        if(finalDF['tideheight1'][i] > finalDF['tideheight2'][i]):
            finalDF['tidetype1'][i] = 1
            finalDF['tidetype2'][i] = -1
            if(pd.notnull(finalDF['tideheight3'][i])):
                finalDF['tidetype3'][i] = 1
                if(pd.notnull(finalDF['tideheight4'][i])):
                    finalDF['tidetype4'][i] = -1
                elif(pd.isnull(finalDF['tideheight4'][i])):
                    finalDF['tideheight4'][i] = 0
            elif(pd.isnull(finalDF['tideheight3'][i])):
                finalDF['tidetype3'][i] = 0
                finalDF['tidetype4'][i] = 0
        elif(finalDF['tideheight1'][i] < finalDF['tideheight2'][i]):
            finalDF['tidetype1'][i] = -1
            finalDF['tidetype2'][i] = 1
            if(pd.notnull(finalDF['tideheight3'][i])):
                finalDF['tidetype3'][i] = -1
                if(pd.notnull(finalDF['tideheight4'][i])):
                    finalDF['tidetype4'][i] = 1
                elif(pd.isnull(finalDF['tideheight4'][i])):
                    finalDF['tideheight4'][i] = 0
            elif(pd.isnull(finalDF['tideheight3'][i])):
                finalDF['tidetype3'][i] = 0
                finalDF['tidetype4'][i] = 0

<ipython-input-28-fd88d118c285>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetype1'][i] = -1
<ipython-input-28-fd88d118c285>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetype2'][i] = 1
<ipython-input-28-fd88d118c285>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF['tidetype3'][i] = 0
<ipython-input-28-fd88d118c285>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [29]:
finalDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 0 to 1460
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1461 non-null   datetime64[ns]
 1   day                   1461 non-null   object        
 2   sunrise               1461 non-null   datetime64[ns]
 3   sunset                1461 non-null   datetime64[ns]
 4   moonrise              1461 non-null   datetime64[ns]
 5   moonset               1461 non-null   datetime64[ns]
 6   distance              1461 non-null   int64         
 7   tidetime1             1461 non-null   datetime64[ns]
 8   tideheight1           1461 non-null   float64       
 9   tidetime2             1411 non-null   datetime64[ns]
 10  tideheight2           1411 non-null   float64       
 11  tidetime3             811 non-null    datetime64[ns]
 12  tideheight3           811 non-null    float64       
 13  tidetime4         

In [30]:
finalDF

,date,day,sunrise,sunset,moonrise,moonset,distance,tidetime1,tideheight1,tidetime2,...,tidetime4,tideheight4,coefficient,coeff_interpretation,redtide_pp,redtide_honda,tidetype1,tidetype2,tidetype3,tidetype4
0,2017-01-01,Sunday,2017-01-01 06:21:00,2017-01-01 17:56:00,2017-01-01 08:29:00,2017-01-01 20:31:00,387620,2017-01-01 07:44:00,-0.2,2017-01-01 23:44:00,...,NaT,NaN,78,high,0,0,-1,1,0,0
1,2017-01-02,Monday,2017-01-02 06:22:00,2017-01-02 17:57:00,2017-01-02 09:16:00,2017-01-02 21:23:00,382892,2017-01-02 08:12:00,-0.2,NaT,...,NaT,NaN,73,high,0,0,-1,0,0,0
2,2017-01-03,Tuesday,2017-01-03 06:22:00,2017-01-03 17:57:00,2017-01-03 10:02:00,2017-01-03 22:14:00,378186,2017-01-03 00:20:00,1.3,2017-01-03 08:47:00,...,NaT,NaN,68,average,0,0,1,-1,0,0
3,2017-01-04,Wednesday,2017-01-04 06:23:00,2017-01-04 17:58:00,2017-01-04 10:48:00,2017-01-04 23:06:00,373755,2017-01-04 00:58:00,1.2,2017-01-04 09:20:00,...,NaT,NaN,62,average,0,0,1,-1,0,0
4,2017-01-05,Thursday,2017-01-05 06:23:00,2017-01-05 17:58:00,2017-01-05 11:34:00,2017-01-05 23:59:00,369848,2017-01-05 01:40:00,1.1,2017-01-05 09:50:00,...,2017-01-05 20:17:00,0.4,58,average,0,0,1,-1,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2020-12-27,Sunday,2020-12-27 06:19:00,2020-12-27 17:53:00,2020-12-27 15:36:00,2020-12-27 03:39:00,403274,2020-12-27 03:51:00,0.0,2020-12-27 10:10:00,...,2020-12-27 20:20:00,1.2,67,average,1,1,-1,1,-1,1
1457,2020-12-28,Monday,2020-12-28 06:20:00,2020-12-28 17:54:00,2020-12-28 16:22:00,2020-12-28 04:29:00,400801,2020-12-28 04:34:00,-0.1,2020-12-28 20:51:00,...,NaT,NaN,72,high,1,1,-1,1,0,0
1458,2020-12-29,Tuesday,2020-12-29 06:20:00,2020-12-29 17:54:00,2020-12-29 17:12:00,2020-12-29 05:21:00,397535,2020-12-29 05:14:00,-0.2,2020-12-29 21:24:00,...,NaT,NaN,76,high,1,1,-1,1,0,0
1459,2020-12-30,Wednesday,2020-12-30 06:21:00,2020-12-30 17:55:00,2020-12-30 18:05:00,2020-12-30 06:14:00,393608,2020-12-30 05:54:00,-0.2,2020-12-30 21:59:00,...,NaT,NaN,78,high,1,1,-1,1,0,0


# EDA

# Scratch Code

Test scrape for one month for Tide Information.

Test scrape for one month for Moonrise and Moonset.